In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values

from board_utils import *

In [2]:
assets_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
single_path = os.path.join(assets_path, 'caissa_master.pgn')
total_games = 619384

#games = load_pgns(single_path, total_games)
#games = [game for game in games if len(str(game.mainline_moves())) > 0]
#print(f'Pre-arranged draw rate: {len(games)/total_games:.2f}%')
#print(f'Total Games: {len(games)}')

In [3]:
def get_keys_func(board_vec):
    self_material_count = int(sum(board_vec[780:785] * [0,6,6,10,9]))
    opp_material_count = int(sum(board_vec[785:790] * [0,6,6,10,9]))
    diff_material_count = self_material_count - opp_material_count
    self_castle_available = int(sum(board_vec[768:770]) > 0)
    opp_castle_available = int(sum(board_vec[770:772]) > 0)
    position_in_endgame = int(sum(board_vec[780:790] * [0,2,2,2,1,0,2,2,2,1]) < 6)
    return self_material_count, opp_material_count, diff_material_count, self_castle_available, opp_castle_available, position_in_endgame

In [4]:
conn = psycopg2.connect(
    dbname="capstone",
    user="postgres",
    password="postgres",
)

cur = conn.cursor()

In [5]:
def insert_or_get_player_id(player_details):
    cur.execute("SELECT player_id FROM players WHERE player_fide_id = %s OR player_name = %s", 
                (player_details['fide_id'], player_details['name']))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO players (player_fide_id, player_name) VALUES (%s, %s) RETURNING player_id", 
                    (player_details['fide_id'], player_details['name']))
        conn.commit()
        return cur.fetchone()[0]

def make_capstone_table():
    create_table_positions = """
    CREATE TABLE IF NOT EXISTS positions (
        position_id SERIAL,
        game_id INT,
        self_material_count INT,
        opp_material_count INT,
        diff_material_count INT,
        self_castle_available INT,
        opp_castle_available INT,
        position_in_endgame INT,
        vector_int INT[],
        vector_float REAL[],
        PRIMARY KEY(position_id)
    )
    """

    create_table_players = """
    CREATE TABLE IF NOT EXISTS players (
        player_id SERIAL,
        player_fide_id INT,
        player_name VARCHAR(256),
        PRIMARY KEY(player_id)
    )
    """

    create_table_game_details = """
    CREATE TABLE IF NOT EXISTS games (
        game_id SERIAL,
        white_id INT,
        black_id INT,
        white_elo INT,
        black_elo INT,
        white_title VARCHAR(16),
        black_title VARCHAR(16),
        event VARCHAR(256),
        site VARCHAR(256),
        date VARCHAR(64),
        round VARCHAR(64),
        result VARCHAR(16),
        eco VARCHAR(16),
        opening VARCHAR(256),
        variation VARCHAR(256),
        PRIMARY KEY(game_id)
    )
    """

    cur.execute("DROP TABLE IF EXISTS positions")
    cur.execute("DROP TABLE IF EXISTS players")
    cur.execute("DROP TABLE IF EXISTS games")
    cur.execute(create_table_positions)
    cur.execute(create_table_players)
    cur.execute(create_table_game_details)

    for game_count, game in enumerate(games):
        metadata_dict = dict(game.headers)
        game_matrix = game_to_csr_matrix(game, csr=False)

        cur.execute("""
        INSERT INTO games (event, site, date, round, result, eco, opening, variation, white_elo, black_elo, white_title, black_title) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING game_id
        """, (
            metadata_dict.get('Event'), metadata_dict.get('Site'), metadata_dict.get('Date'),
            metadata_dict.get('Round'), metadata_dict.get('Result'), metadata_dict.get('ECO'),
            metadata_dict.get('Opening'), metadata_dict.get('Variation'), metadata_dict.get('WhiteElo'),
            metadata_dict.get('BlackElo'), metadata_dict.get('WhiteTitle'), metadata_dict.get('BlackTitle')
        ))
        game_id = cur.fetchone()[0]
        
        white_id = insert_or_get_player_id({'fide_id': metadata_dict.get('WhiteFideId'), 'name': metadata_dict['White']})
        black_id = insert_or_get_player_id({'fide_id': metadata_dict.get('BlackFideId'), 'name': metadata_dict['Black']})
        cur.execute("UPDATE games SET white_id = %s, black_id = %s WHERE game_id = %s", (white_id, black_id, game_id))

        integer_matrix = game_matrix[:, :780]
        float_matrix = game_matrix[:, 780:]
        
        position_records = []
        for i in range(len(game_matrix)):
            position_vars = get_keys_func(np.concatenate([integer_matrix[i], float_matrix[i]]))
            position_record = (game_id, *position_vars, list(integer_matrix[i]), list(float_matrix[i]))
            position_records.append(position_record)

        execute_values(cur, """
        INSERT INTO positions (game_id, self_material_count, opp_material_count, diff_material_count, 
        self_castle_available, opp_castle_available, position_in_endgame, vector_int, vector_float) 
        VALUES %s
        """, position_records)
        if game_count % 64 == 0:
            conn.commit()
    conn.commit()

In [6]:
def clear_column(colname='pgn_string', data="VARCHAR(4096)", cur=cur, conn=conn):
    query = f"ALTER TABLE games DROP COLUMN {colname}"
    cur.execute(query)
    query = f"ALTER TABLE games ADD COLUMN {colname} {data}"
    cur.execute(query)
    conn.commit()

In [7]:
def pgn_into_capstone_table():
    full_pgn_gen = (str(g) for g in games)
    clear_column('pgn_string')
    game_id = 0

    for pgn_string in full_pgn_gen:
        game_id += 1
        insert_pgn = """UPDATE games SET pgn_string = %s WHERE game_id = %s"""
        cur.execute(insert_pgn, (pgn_string, game_id))

        if game_id%100==0:
            conn.commit()
    conn.commit()

In [8]:
#make_capstone_table() # 30 hours
#pgn_into_capstone_table() # 30 minutes

In [9]:
query = """
WITH combined_ids AS (
    SELECT black_id AS player_id FROM games
    UNION ALL
    SELECT white_id AS player_id FROM games
),
frequency AS (
    SELECT player_id, COUNT(*) AS freq
    FROM combined_ids
    GROUP BY player_id
)
SELECT p.player_name, f.freq
FROM frequency f
JOIN players p ON p.player_id = f.player_id
ORDER BY f.freq DESC LIMIT 20;
"""

cur.execute(query)
matches = cur.fetchall()
print("All matches in the database:")
for match in matches:
    print(match)

#self_material_count, opp_material_count, diff_material_count, self_castle_available, opp_castle_available, position_in_endgame
#query_fen = 'rnb2b1r/2Nnkppp/p2pp3/8/PPBN1P2/2p3Pq/4Q2P/R3K1R1 w Q - 1 17' #wild opening
#query_fen = 'rnbq1rk1/pp2ppbp/3p1np1/8/3NP3/2N1BP2/PPP3PP/R2QKB1R w KQ - 1 8' #dragon
#query_fen = 'r1bq1rk1/ppp2pbp/2np1np1/4p3/2PPP3/2N2N2/PP2BPPP/R1BQ1RK1 w - - 2 8' #kid
#query_fen = '6k1/5p2/1r2p1p1/3p1n2/3P3p/1P1R3P/4NPP1/6K1 b - - 2 36' #knight rook endgame
#query_fen = 'r4rk1/ppqn1ppp/3bpn2/3pN3/3P4/2P2N2/PP2QPPP/R1B1R1K1 b - - 4 12' #carlsbad caro
query_fen = 'r1b1r1k1/pp1nqppp/2p5/3p4/3Pn3/2NBPN2/PPQ2PPP/R4RK1 w - - 2 12' #carlsbad
query_array = board_to_array(chess.Board(fen=query_fen))
query_array = orient_board(query_array, query_fen.split()[1]=='w')

keys = (8, 8, 0, 1, 1, 1) #knightrookend
#keys = get_keys_func(query_array) #(28, 28, 0, 0, 0, 0) #carlsbad

sm, om, dm, sc, oc, pe = keys

query = f'''SELECT position_id FROM positions
WHERE self_material_count < {sm+5.5} AND self_material_count > {sm-5.5} 
AND opp_material_count < {om+5.5} AND opp_material_count > {om-5.5} 
AND diff_material_count < {dm+1.5} AND diff_material_count > {dm-1.5} 
AND (self_castle_available = {sc} OR opp_castle_available = {oc})
AND position_in_endgame = {pe}'''

cur.execute(query)
matches = cur.fetchall()
print()
print('Number of matches:')
print(len(matches))

All matches in the database:
('Nakamura, Hi', 4679)
('Sarana, A', 3526)
('Andreikin, D', 3277)
('Kamsky, G', 3188)
('Carlsen, M', 3151)
('Jobava, Ba', 3063)
('Fedoseev, Vl3', 3007)
('Vachier Lagrave, M', 2968)
('Mamedyarov, S', 2933)
('Kosteniuk, A', 2928)
('Grischuk, A', 2891)
('Dubov, Daniil', 2810)
('Xiong, Jeffery', 2806)
('Zhigalko, S', 2800)
('So, W', 2757)
('Sjugirov, S', 2746)
('Chigaev, M', 2731)
('Bortnyk, Olexandr', 2691)
('Paravyan, D', 2662)
('Caruana, F', 2661)

Number of matches:
4927


In [10]:
cur.close()
conn.close()